# What happens if we have a Whale Attack?

The Malicious Whale Test:
“Moby Dick” buys up a large portion of the RAI supply and uses it to forcibly hold the market price at a constant, we consider 4 controller types and verify that the whale expects to LOSE
grid 2 x 2
positive vs negative Ki
leaky integral vs no leaky integral
result
all tests pass except -Ki w/ no leak

## Beginning of new work

In [1]:
%load_ext autotime

time: 1.01 ms (started: 2021-04-02 15:43:31 -03:00)


In [2]:
# Dependences
import pandas as pd
import numpy as np
import inspect

# Experiments
from model import run
from model import sys_params 
import plotly.express as px
df = run.run()
#rdf = run.postprocessing(df)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 5
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1000, 9, 1, 7)



Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0]
SubsetIDs: [0, 1, 2, 3, 4]
Ns       : [0, 1, 2, 3, 4]
ExpIDs   : [0, 0, 0, 0, 0]
Execution Mode: parallelized


/home/danlessa/repos/bsci/reflexer/notebooks/analysis/whale-test/model/parts/system.py:21: RuntimeWarning: overflow encountered in double_scalars
  target_price = s['target']*(1+s['price_adjustment_rate'])**Dt
/home/danlessa/repos/bsci/reflexer/notebooks/analysis/whale-test/model/parts/system.py:122: RuntimeWarning: invalid value encountered in double_scalars
  continuous_rate = params['Kp']*s['error']+params['Ki']*s['integral']



Total execution time: 3.07s
time: 7.52 s (started: 2021-04-02 15:43:31 -03:00)


In [3]:
ldf= df[df.substep==df.substep.max()]

time: 45.7 ms (started: 2021-04-02 15:43:39 -03:00)


In [4]:
ldf.head()

,price,target,error,integral,Dt,timestamp,price_adjustment_rate,simulation,subset,run,substep,timestep
3,4,3.493704,-0.506296,-1812.288501,3601.0,2021-04-02 16:43:33.795045,-5.012619e-07,0,0,1,3,1
6,4,3.487374,-0.512626,-3482.235227,3618.0,2021-04-02 17:43:52.660718,-5.029535e-07,0,0,1,3,2
9,4,3.481044,-0.518956,-5012.840150,3612.0,2021-04-02 18:44:04.701596,-5.050315e-07,0,0,1,3,3
12,4,3.474708,-0.525292,-6418.217655,3607.0,2021-04-02 19:44:12.349540,-5.074631e-07,0,0,1,3,4
15,4,3.468301,-0.531699,-7723.606342,3637.0,2021-04-02 20:44:50.121079,-5.102442e-07,0,0,1,3,5


time: 72.8 ms (started: 2021-04-02 15:43:39 -03:00)


In [5]:
ldf.subset.unique()

array([0, 1, 2, 3, 4])

time: 8.43 ms (started: 2021-04-02 15:43:39 -03:00)


In [6]:
ldf=ldf.dropna()
endtimes = ldf[ldf.subset==2].timestep.max()

time: 16.7 ms (started: 2021-04-02 15:43:39 -03:00)


In [7]:
endtimes

348

time: 6.63 ms (started: 2021-04-02 15:43:39 -03:00)


In [8]:
cutoff = 320
ldf['drop'] = (ldf.subset.apply(lambda x: x==2))&(ldf.timestep.apply(lambda x: x> cutoff))

ldf = ldf[~ldf['drop']].copy()

time: 20.3 ms (started: 2021-04-02 15:43:39 -03:00)


In [40]:
subset_map = {0: 'Negative ki, with leak',
              1: 'Positive ki, with leak',
              2: 'Negative ki, without leak',
              3: 'Positive ki, without leak',
              4: 'Zero ki'}


fig_df = (ldf.assign(Scenario=ldf.subset.map(subset_map)))

px.line(fig_df,
        x='timestamp',
        y='price_adjustment_rate',
        color='Scenario',
                  width=800,
          height=400,
        title='Hourly Redemption Rate for different integrator parameters',
        labels={'price_adjustment_rate': 'Redemption Rate (% / hr)',
                'timestamp': 'Time'})

time: 1.27 s (started: 2021-04-02 15:56:01 -03:00)


In [41]:
fig = px.line(fig_df,
              x='timestamp',
              y='target',
              color='Scenario',              width=800,
              height=400,

              title='Redemption Price for different integrator parameters',
              labels={'target': 'Redemption Price (RAI / USD)',
                      'timestamp': 'Time'})

fig.show()

time: 895 ms (started: 2021-04-02 15:56:14 -03:00)


In [44]:
fig = px.line(fig_df,
              x='timestamp',
              y='error',
              color='Scenario',              width=800,
              height=400,

              title='Error (difference between redemption and market prices) per time',
              labels={'error': 'Error (RAI / USD)',
                      'timestamp': 'Time'})

fig.show()

time: 876 ms (started: 2021-04-02 15:58:35 -03:00)


In [47]:
fig = px.line(fig_df,
              x='timestamp',
              y='integral',
              color='Scenario',              width=800,
              height=400,

              title='Error (difference between redemption and market prices) per time',
              labels={'error': 'Error (RAI / USD)',
                      'timestamp': 'Time'})

fig.show()

time: 770 ms (started: 2021-04-02 16:00:23 -03:00)


In [ ]:
ldf[ldf.subset==0].plot(x='timestamp', y='integral')
ax = plt.gca()
ldf[ldf.subset==1].plot(x='timestamp', y='integral',ax=ax)
ldf[ldf.subset==2].plot(x='timestamp', y='integral',  ax=ax)
ldf[ldf.subset==3].plot(x='timestamp', y='integral', ax=ax)
plt.legend(['negative ki w/ leak', 'positive ki w/ leak', 'negative ki w/o leak', 'positive ki w/o leak'])

In [ ]:
sdf = ldf[ldf.timestep <50].copy()

In [ ]:
sdf[sdf.subset==0].plot(x='timestamp', y='price_adjustment_rate')
ax = plt.gca()
sdf[sdf.subset==1].plot(x='timestamp', y='price_adjustment_rate', ax=ax)
sdf[sdf.subset==2].plot(x='timestamp', y='price_adjustment_rate', ax=ax)
sdf[sdf.subset==3].plot(x='timestamp', y='price_adjustment_rate', ax=ax)
plt.legend(['negative ki w/ leak', 'positive ki w/ leak', 'negative ki w/o leak', 'positive ki w/o leak'])

In [ ]:
sdf[sdf.subset==0].plot(x='timestamp', y='target', logy=True)
ax = plt.gca()
sdf[sdf.subset==1].plot(x='timestamp', y='target', logy=True,ax=ax)
sdf[sdf.subset==2].plot(x='timestamp', y='target', logy=True, ax=ax)
sdf[sdf.subset==3].plot(x='timestamp', y='target', logy=True, ax=ax)
plt.legend(['negative ki w/ leak', 'positive ki w/ leak', 'negative ki w/o leak', 'positive ki w/o leak'])

In [ ]:
sdf[sdf.subset==0].plot(x='timestamp', y='error')
ax = plt.gca()
sdf[sdf.subset==1].plot(x='timestamp', y='error',ax=ax)
sdf[sdf.subset==2].plot(x='timestamp', y='error',  ax=ax)
sdf[sdf.subset==3].plot(x='timestamp', y='error', ax=ax)
plt.legend(['negative ki w/ leak', 'positive ki w/ leak', 'negative ki w/o leak', 'positive ki w/o leak'])

In [ ]:
sdf[sdf.subset==0].plot(x='timestamp', y='integral')
ax = plt.gca()
sdf[sdf.subset==1].plot(x='timestamp', y='integral',ax=ax)
sdf[sdf.subset==2].plot(x='timestamp', y='integral',  ax=ax)
sdf[sdf.subset==3].plot(x='timestamp', y='integral', ax=ax)
plt.legend(['negative ki w/ leak', 'positive ki w/ leak', 'negative ki w/o leak', 'positive ki w/o leak'])

## end of my new work

In [ ]:
from model.run import postprocessing
function = inspect.getsource(postprocessing)
print(function)

## Analyze Result

Below is the refined pandas dataframe

In [ ]:
rdf.tail()

In [ ]:
plt.figure(figsize=(18, 12))
rdf.plot(x='timestamp', y=['normed_price','normed_target'], marker='.', ax=plt.gca())

In the above plot, we show our martingle process or random walk, for price and compare to the target, which is stable. Below we show our martingale process code, as implemented in our cadCAD code. We normalize the price and target by dividing the price or target by TOK, which is 10**27.

In [ ]:
from model.parts.system import martingale
function = inspect.getsource(martingale)
print(function)

In [ ]:
plt.figure(figsize=(18, 12))
rdf.plot(x='timestamp', y=['normed_integral','normed_error'], marker='.', ax=plt.gca())

In [ ]:
plt.figure(figsize=(18, 12))
rdf[['normed_new_error','normed_old_error','timestamp']].plot(x='timestamp', marker='>', ax=plt.gca())
rdf[['normed_mean_error','timestamp']].plot(x='timestamp', marker=None, linestyle='--', ax=plt.gca())

In [ ]:
rdf['seconds'] = rdf.Dt.cumsum()


x=rdf.seconds.values
y=rdf.normed_error.values
h=rdf.normed_mean_error.values
w=rdf.Dt.values

plt.figure(figsize=(18, 12))
plt.plot(x,y , marker='.',linestyle='--', color='r')

ax = plt.gca()

for p in range(len(x)-1):
    ax.add_patch(
        Rectangle(
        (x[p], 0),
        w[p+1],
        h[p+1],
        edgecolor = 'blue',
        facecolor = 'cyan',
        fill=True
     ) )

plt.title('Visualizing Numerical Integration')
plt.xlabel('Time passed in seconds')
plt.ylabel('Observed Error in 1/RAY')


## Conclusion
In this notebook, we've provided an overview of integration and numerical integration, created a cadCAD model to illustrate numerical integration, ran the simulation, plotted and analysed the results.